<a href="https://colab.research.google.com/github/ELehmann91/FS1/blob/master/Homework_DL_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Todos

### # (1) You predict one week into the future now, instead of 1 day. Predicting on a more aggregate level removes noise and will thus increase prediction accuracy.

# Predicting Blabla

## Dataset

[Beijing PM2.5 Data Data Set](https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data)

### Columns of the dataset:

# Description of the data files

*All data has been anonymized. You must nevertheless not share this data-set with any outsiders.*

* The information is contained in two datafiles:
    * [*solditems_encoded_stage2.csv* contains sales information](#information-on-product-sales)
    * [*content_encoded_stage2.csv* contains information on product features](#information-on-the-properties-of-the-products)
* Note that not all variables were originally specified for each column, but there are no NA (missing) values in the datafiles:
    * For categorical variables, missing values include a 0 in all one-hot encoded columns of the variable.
    * For numerical variables, missing values were simply filled with the column mean (the most simple imputation).

## Information on product sales

1) Datafile: **solditems_encoded_stage2.csv**
2) Each row corresponds to the sale of one specific product token.
3) Apart from the first two columns,
    * each *numerical variable* has been normalized to the [0,1] interval;
    * each *categorical variable* (including *binary variables*) has been one-hot encoded. 

Description of the columns:

* `product_sid`: The unique identifier of a product model. You can think of this as the type, and the rows of the sales data correspond to tokens of these types.
    * This identifier corresponds to the `ProductId` in the datafile describing properties of product models.
* `created_date_id`: This is the date in 'YYYY-MM-DD' format when the specific product token was sold.
* `sales_item_price_created`: The actual price at which the product token was sold.
* `sales_item_price`:  The price of the product token.
* `sales_voucher_created`: The actual price of a voucher for the product token.
* `sales_voucher`: The price of a voucher for the product token.
* `sales_value_created`: The actual price paid by the buyer.
* `sales_value`: The price to be payed by the buyer.
* `created_year`: The year of the sale.
* `created_month`: The month of the sale.
* `created_weekday`: The weekday of the sale *originally* encoded from 0 (Monday) to 6 (Sunday).
* `days_since_first_sold`: The number of days since the frst sale happened
* `days_since_release`: The number of days since the product was released.
* `Ratio_SalesItemPrice_per_created`: Ratio of the *sales_item_price / sales_item_price_created* variables.
* `Ratio_SalesValue_per_created`:  Ratio of the *sales_value / sales_value_created* variables.
* `Ratio_SalesItemPrice_per_SalesValue`: Ratio of the *sales_item_price / sales_value* variables.
* `Ratio_SalesItemPriceCreated_per_SalesValueCreated`: Ratio of the *sales_item_price_created / sales_value_created* variables.
* `channel_sid_N`: One-hot encoded column of the *channel_sid* variable which specifies the channel (e.g., web browser) on which the sale happened.
* `returned_date_id_N`:  One-hot encoded column of the *returned_date* variable which specifies if a product token was returned (N=1) or not (N=0).

Note that the ratio values were included so that the original relation between the variables in question is not lost after scaling the columns.

Note also that some of the columns have been pre-computed for your convenience only; if you like, you can compute them or other derived variables yourself.

## Information on the properties of the products

1) Datafile: **content_encoded_stage2.csv**
2) Each row corresponds to a product model (that is, a type whose tokens are rows of the sales datafile).
3) Apart from the first two columns,
    * each *numerical variable* has been normalized to the [0,1] interval;
    * each *categorical variable* (including *binary variables*) has been one-hot encoded. 

Description of the columns:

* `ProductId`: The unique identifier of a product model.
    * This identifier corresponds to the `product_sid` in the datafile describing product sales.
* `ProductNameCleaned`: the product name without the color specification (encoded here as an integer). 
This means that several product models (with different product ID's) may be assigned to the same ProductNameCleaned.  
* `ReleaseYear`, `ReleaseMonth`, `ReleaseDayOfWeek` encode the year, month and weekday (originally from 0=Monday to 6=Sunday) of the date when the product was released.
* `DisplaySize_in`, `Storage_GB`, `CPUSpeed_GHz`, `DisplayHeight_px`, `DisplayResolution_MP`, `DisplayWidth_px`, `FrontCamResolution_MP`, 
`MainDescription_NumOfWords`, `Network_NumOfFreqs`, `RAM_GB` , `RearCamResolution_MP` , 
`Size_Depth` , `Size_Height` , `Size_Width` , `Weight` , `Ratio_HeightPerWidth` , `Ratio_DepthPerWidth` 
are numerical variables describing different properties of a product. Most of these are self-explanatory except maybe for the following:
    * `MainDescription_NumOfWords` encoded the number of words in the textual description of the product.
    * `Network_NumOfFreqs` encoded the number of frequencies the product is capable of handling.
* The rest of the columns are one-hot-encoded columns of categorical variables capturing different properties of the products. Again, the names of the variables should be self-explanatory.
    d direction -> Todo: Not that good

**Iws:** Cumulated wind speed -> Todo: Not that good

**Is:** Cumulated hours of snow

**Ir:** Cumulated hours of rain

In [0]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pollution.csv

In [0]:
#!pip install seglearn

In [0]:
import pandas as pd 
import csv
import numpy as np 
import matplotlib.pyplot as plt

#import warnings
#warnings.filterwarnings("ignore")

In [0]:
def sniff_format(location):
    with open(location, newline='') as csvfile:
        sniffer = csv.Sniffer()
        sample = csvfile.read(1024)
        dialect = sniffer.sniff(sample)
        header = sniffer.has_header(sample)
        if header:
            header=0
        else:
            header=None

    return {"dialect":dialect, "header":header}

def describe_full(df):
    #pd.options.display.float_format = '{:.2f}'.format
    dtypes_description=pd.DataFrame(dict(df.dtypes),["dtypes"])
    na_description = pd.DataFrame(dict(df.isna().sum()),["NA-s"])
    na_percent = ((pd.DataFrame(dict(df.isna().sum()),["NA%"])/len(df))*100).round(decimals=2)
    description = df.describe(include='all')
    full_description = dtypes_description.append(na_description).append(na_percent).append(description).replace(np.nan, '', regex=True)

    mask = full_description.loc["freq",:]==1
    full_description.at[["top"],mask.index[mask]]=""
    #TODO: scientific notation - could be nicer
    
    return full_description

In [0]:
link_data_solditems = "https://drive.google.com/open?id=1Fi0Rkloif2hC73FelAMwKOJRDely7EG4"
fluff_data_solditems, id_data_solditems = link_data_solditems.split('=')

downloaded_data_solditems = drive.CreateFile({'id':id_data_solditems}) 
downloaded_data_solditems.GetContentFile('solditems_encoded_stage2.csv')  
df_data_solditems = pd.read_csv('solditems_encoded_stage2.csv')

#print(df_data_solditems.head())

In [0]:
link_data_content = "https://drive.google.com/open?id=1PgSjV_xgycDd2CDdBAV_maSfJesiQJgH"
fluff_data_content, id_data_content = link_data_content.split('=')

downloaded_data_content = drive.CreateFile({'id':id_data_content}) 
downloaded_data_content.GetContentFile('content_encoded_stage2.csv')  
df_data_content = pd.read_csv('content_encoded_stage2.csv')

#print(df_data_content.head(1))

In [0]:
data = df_data_solditems.merge(df_data_content, left_on='product_sid', right_on='ProductId')

In [0]:
print(len(df_data_solditems),len(df_data_content))

547248 3656


In [0]:
for col in list(df_data_solditems):
  print('items:',col)


items: product_sid
items: created_date_id
items: sales_item_price_created
items: sales_item_price
items: sales_voucher_created
items: sales_voucher
items: sales_value_created
items: sales_value
items: created_year
items: created_month
items: created_weekday
items: days_since_first_sold
items: days_since_release
items: Ratio_SalesItemPrice_per_created
items: Ratio_SalesValue_per_created
items: Ratio_SalesItemPrice_per_SalesValue
items: Ratio_SalesItemPriceCreated_per_SalesValueCreated
items: channel_sid_0
items: channel_sid_1
items: channel_sid_2
items: returned_date_id_0
items: returned_date_id_1


In [0]:
# These are manufacturers where a lot of data is available, which, again, reduces noise and makes prediction easier.
# df_data_content.drop(columns=['Manufacturer_-1', 'Manufacturer_1', 'Manufacturer_2', 'Manufacturer_4', 'Manufacturer_5',
#                  'Manufacturer_6', 'Manufacturer_7', 'Manufacturer_8', 'Manufacturer_9', 'Manufacturer_10',
#                  'Manufacturer_11', 'Manufacturer_12', 'Manufacturer_13', 'Manufacturer_15', 'Manufacturer_16',
#                  'Manufacturer_17', 'Manufacturer_18', 'Manufacturer_19', 'Manufacturer_20', 'Manufacturer_21',
#                  'Manufacturer_22', 'Manufacturer_23', 'Manufacturer_24', 'Manufacturer_25', 'Manufacturer_26',
#                  'Manufacturer_27', 'Manufacturer_28', 'Manufacturer_29', 'Manufacturer_30', 'Manufacturer_31',
#                  'Manufacturer_33', 'Manufacturer_34', 'Manufacturer_35', 'Manufacturer_36', 'Manufacturer_37',
#                  'Manufacturer_38', 'Manufacturer_39', 'Manufacturer_40', 'Manufacturer_41', 'Manufacturer_42',
#                  'Manufacturer_43', 'Manufacturer_44', 'Manufacturer_45', 'Manufacturer_46', 'Manufacturer_47',
#                  'Manufacturer_48', 'Manufacturer_49', 'Manufacturer_50', 'Manufacturer_51', 'Manufacturer_52',
#                  'Manufacturer_53', 'Manufacturer_54', 'Manufacturer_55', 'Manufacturer_56', 'Manufacturer_57',
#                  'Manufacturer_58', 'Manufacturer_59', 'Manufacturer_60', 'Manufacturer_61', 'Manufacturer_62',
#                  'Manufacturer_63', 'Manufacturer_64', 'Manufacturer_65', 'Manufacturer_66', 'Manufacturer_67',
#                  'Manufacturer_68', 'Manufacturer_69', 'Manufacturer_70', 'Manufacturer_71', 'Manufacturer_72',
#                  'Manufacturer_73', 'Manufacturer_74'], 
#         inplace=True)

# for col2 in list(df_data_content):
#     print('content',col2)

In [0]:
df_data_content['ProductId'][:10]

0    0
1    1
2    2
3    3
4    4
5    5
6    6
7    7
8    8
9    9
Name: ProductId, dtype: int64

In [0]:
len(df_data_content)

3656

In [0]:
df_data_content.apply(lambda x: x.count(), axis=1)

0       848
1       848
2       848
3       848
4       848
5       848
6       848
7       848
8       848
9       848
10      848
11      848
12      848
13      848
14      848
15      848
16      848
17      848
18      848
19      848
20      848
21      848
22      848
23      848
24      848
25      848
26      848
27      848
28      848
29      848
       ... 
3626    848
3627    848
3628    848
3629    848
3630    848
3631    848
3632    848
3633    848
3634    848
3635    848
3636    848
3637    848
3638    848
3639    848
3640    848
3641    848
3642    848
3643    848
3644    848
3645    848
3646    848
3647    848
3648    848
3649    848
3650    848
3651    848
3652    848
3653    848
3654    848
3655    848
Length: 3656, dtype: int64

In [0]:
data_new = data_new.dropna()
len(data_new)


1377

In [0]:
import numpy as np
data_new = df_data_content[['Manufacturer_-1', 'Manufacturer_1', 'Manufacturer_2', 'Manufacturer_4', 'Manufacturer_5',
                 'Manufacturer_6', 'Manufacturer_7', 'Manufacturer_8', 'Manufacturer_9', 'Manufacturer_10',
                 'Manufacturer_11', 'Manufacturer_12', 'Manufacturer_13', 'Manufacturer_15', 'Manufacturer_16',
                 'Manufacturer_17', 'Manufacturer_18', 'Manufacturer_19', 'Manufacturer_20', 'Manufacturer_21',
                 'Manufacturer_22', 'Manufacturer_23', 'Manufacturer_24', 'Manufacturer_25', 'Manufacturer_26',
                 'Manufacturer_27', 'Manufacturer_28', 'Manufacturer_29', 'Manufacturer_30', 'Manufacturer_31',
                 'Manufacturer_33', 'Manufacturer_34', 'Manufacturer_35', 'Manufacturer_36', 'Manufacturer_37',
                 'Manufacturer_38', 'Manufacturer_39', 'Manufacturer_40', 'Manufacturer_41', 'Manufacturer_42',
                 'Manufacturer_43', 'Manufacturer_44', 'Manufacturer_45', 'Manufacturer_46', 'Manufacturer_47',
                 'Manufacturer_48', 'Manufacturer_49', 'Manufacturer_50', 'Manufacturer_51', 'Manufacturer_52',
                 'Manufacturer_53', 'Manufacturer_54', 'Manufacturer_55', 'Manufacturer_56', 'Manufacturer_57',
                 'Manufacturer_58', 'Manufacturer_59', 'Manufacturer_60', 'Manufacturer_61', 'Manufacturer_62',
                 'Manufacturer_63', 'Manufacturer_64', 'Manufacturer_65', 'Manufacturer_66', 'Manufacturer_67',
                 'Manufacturer_68', 'Manufacturer_69', 'Manufacturer_70', 'Manufacturer_71', 'Manufacturer_72',
                 'Manufacturer_73', 'Manufacturer_74']].replace(1,np.nan)

In [0]:
df = df_data_solditems.merge(df_data_content, left_on='product_sid', right_on='ProductId')

In [0]:
print(df.columns)
print(len(df.columns))

Index(['product_sid', 'created_date_id', 'sales_item_price_created',
       'sales_item_price', 'sales_voucher_created', 'sales_voucher',
       'sales_value_created', 'sales_value', 'created_year', 'created_month',
       ...
       'OS_5', 'OS_6', 'OS_7', 'OS_8', 'Radio_0', 'Radio_1', 'UMTS_0',
       'UMTS_1', 'WLAN_0', 'WLAN_1'],
      dtype='object', length=870)
870


In [0]:
# There is a warning that would be worth investigationg, but for now, let's ignore it
# import warnings
# warnings.filterwarnings("ignore")

In [0]:
# df["date"]= pd.to_datetime(df['year'].astype(str)+'-'+df['month'].astype(str)+"-"+df["day"].astype(str)+"T"+df["hour"].astype(str).apply(lambda x: x.zfill(2)+":00"))
# df.set_index(df.date, inplace=True)
# df.drop("date", axis=1, inplace=True)
# df.head(10)



## Exploratory Data Analysis (EDA)

In [0]:
print(len(df.columns))

870


## Encoding day of week

We explicitly encode the day of week, since we assume that weekends and workdays behave differently.

In [0]:
df["dayofweek"]=df.index.dayofweek+1
# Starts with 1 (Monday); usually, Monday is 0. Only 1-7 is in the column.

AttributeError: ignored

# Decision about NaN-s

In [0]:
fig = plt.gcf()
fig.set_size_inches(10,7)
df["pm2.5"].isnull().astype(float).plot()

In [0]:
#df.groupby(df.index.weekday_name)["pm2.5"].isnull().astype(int).sum()
def isnullsum(df):
    return df.isnull().sum()


#print("All datapoints per year:",df.groupby(df.index.year)["pm2.5"].count())
#print("NaN datapoints per year:",df.groupby(df.index.year)["pm2.5"].apply(isnullsum))
print("% NaN datapoints per year:",(df.groupby(df.index.year)["pm2.5"].apply(isnullsum)/df.groupby(df.index.year)["pm2.5"].count())*100.0)

#print("--------------------------------")


#print("All datapoints per month:",df.groupby(df.index.month)["pm2.5"].count())
#print("NaN datapoints per month:",df.groupby(df.index.month)["pm2.5"].apply(isnullsum))
print("% NaN datapoints per year:",(df.groupby(df.index.month)["pm2.5"].apply(isnullsum)/df.groupby(df.index.month)["pm2.5"].count())*100.0)

#print("--------------------------------")


#print("All datapoints per weekday:",df.groupby(df.index.weekday_name)["pm2.5"].count())
#print("NaN datapoints per weekday:",df.groupby(df.index.weekday_name)["pm2.5"].apply(isnullsum))
print("% NaN datapoints per year:",(df.groupby(df.index.weekday_name)["pm2.5"].apply(isnullsum)/df.groupby(df.index.weekday_name)["pm2.5"].count())*100.0)

#print("--------------------------------")

#print("All datapoints per weekday:",df.groupby(df.index.hour)["pm2.5"].count())
#print("NaN datapoints per weekday:",df.groupby(df.index.hour)["pm2.5"].apply(isnullsum))
print("% NaN datapoints per year:",(df.groupby(df.index.hour)["pm2.5"].apply(isnullsum)/df.groupby(df.index.hour)["pm2.5"].count())*100.0)

After examining the NaN values in pm2.5, we see no obvious temporal pattern. This is cause for worry, since by simply dropping the rows with NaN values, we can destroy the temporal coherence of the data, hence **data imputation is desirable.**

The autocorrelation charts below imply, that it is not unreasonable to take the previous value to fill NaN-s (high autocorrelation with the previous timestep).

In [0]:
df.fillna(method='ffill', inplace=True)

print(df.isnull().sum())

df.dropna(inplace=True)

print(df.isnull().sum())

# Examining autocorrelations

In [0]:
from statsmodels.graphics.tsaplots import plot_pacf

#columns = [] #use this for speedup
columns = ["pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

for col in columns:
    plt.figure()
    plot_pacf(df[col].dropna(), lags=200, zero=False)
    
plt.show()


## What do we see?

Well, the fact, that we don't see.

Or more precisely: smog (and weather) is slow to move, it is extremely strongly autocorrelated with itself one-two hours before, so in order to at least be able to see some autocorrelation structure beyond this, we need to filter out the first some hours from our autocorrelation analysis.

(By the way, that's why we don't stick to the prediction of the next hour as in the original "inspiration" blogpost. Would not be too relevant...)

In [0]:
from statsmodels.graphics.tsaplots import _prepare_data_corr_plot, _plot_corr
import statsmodels.graphics.utils as utils
from statsmodels.tsa.stattools import pacf

def plot_pacf_drop(x, ax=None, lags=None, alpha=.05, method='ywunbiased',
              use_vlines=True, title='Partial Autocorrelation', zero=True,
              vlines_kwargs=None, drop_no=0, **kwargs):
    
    lags_orig=lags
    fig, ax = utils.create_mpl_ax(ax)
    vlines_kwargs = {} if vlines_kwargs is None else vlines_kwargs
    lags, nlags, irregular = _prepare_data_corr_plot(x, lags, zero)
    confint = None
    if alpha is None:
        acf_x = pacf(x, nlags=nlags, alpha=alpha, method=method)
    else:
        acf_x, confint = pacf(x, nlags=nlags, alpha=alpha, method=method)

    if drop_no:
        acf_x = acf_x[drop_no+1:]
        confint = confint[drop_no+1:]
        lags, nlags, irregular = _prepare_data_corr_plot(x, lags_orig-drop_no, zero)
        
    _plot_corr(ax, title, acf_x, confint, lags, False, use_vlines,
               vlines_kwargs, **kwargs)

    return fig

In [0]:
import matplotlib.pyplot as plt

#columns = [] #use this for speedup
columns = ["pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

for col in columns:
    plt.figure()
    plot_pacf_drop(df[col].dropna(), lags=200, drop_no=3, zero=False)
    
plt.show()

Studying even the filtered charts leaves us in doubt about the possible window for modeling (in case of the classical models), so we will keep 200 as the modeling window (nearly two weeks). This is a parameter that is worth empirically studying later on.

It is worth mentioning, that the `pacf` charts would definitely change drastically if we would use some differencing. Since down below we decide not to, we keep it as it is.

## Seasonal decomposition and the question of trends

In [0]:
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.tsatools import freq_to_period
import matplotlib.pyplot as plt

analysis = seasonal_decompose(df["pm2.5"], freq=freq_to_period(df.index.inferred_freq))

analysis.plot()
plt.show()

Well, the default setting (infer periods - hourly) is rather uninformative, so it is maybe worth using some domain knowledge here, and use yearly frequency.

In [0]:
analysis = seasonal_decompose(df["pm2.5"], freq=24*365)

analysis.plot()
plt.show()

We do get the first impression, that there is no overarching simple trend, as well as there are non-trivial seasonal patterns. At a later stage we should investigate differencing regimes, but for now, we leave the data as is.

# Train, valid, test split - before normalization

Contamination by the normalization values is a distant possibility, but let's stick to paranoid practices.

In [0]:
VALID_AND_TEST_SIZE=0.1

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_else, y_train, y_else = train_test_split(df, df["pm2.5"], test_size=VALID_AND_TEST_SIZE*2, shuffle=False)
X_valid, X_test, y_valid, y_test = train_test_split(X_else, y_else, test_size=0.5, shuffle=False)

We could have used `temporal_split` from `seglearn`, but that would have cast everything to numpy, so it was more convenient this way for now. Using `seglearn` is encouraged - if we would like to go into classical modeling.

# Data normalization

Our default assumption is to use Scikit's minmax scaler for easier learning by neural models.

But there are some exceptions:

## How to normalize dates?

For the year it is more tricky, it is basically an ordinal.
Subtracting the first year is nice, but how to handle the normalization to 0,1?

We could use 2018 as a max, but **WE WOULD HAVE TO WRITE A BIG CAVEAT MESSAGE FOR DEPLOY PEOPLE!**

So it should be something like  `(df.year - (df.year.min())-1)/((df.year.max()-df.year.min())*2)` (-1 is for avoiding zero, making the life of the network more easy...)

For now we stick to the minmax scaler (living risky... :-)

For month, day, hour default assumption is, scikit's minmax scaler could work, but we will choose a more elaborate solution from [here](https://ianlondon.github.io/blog/encoding-cyclical-features-24hour-time/). This capitalizes on the circular nature of these quasi ordinals.


In [0]:
from sklearn.preprocessing import MinMaxScaler
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
# I literally hate when a standard Scikit function throws big bunches of warnings 
# - though suppressing them is a dangerous practice. Hence this comment. 

def minmax_scale(df_x,series_y, normalizers=None):
    features_to_minmax = ["year","pm2.5","DEWP","TEMP","PRES","Iws","Is","Ir"]

    if not normalizers:
        normalizers = {}

    for feat in features_to_minmax:
        if feat not in normalizers:
            normalizers[feat] = MinMaxScaler()
            normalizers[feat].fit(df_x[feat].values.reshape(-1, 1))
        
        df_x[feat] = normalizers[feat].transform(df_x[feat].values.reshape(-1, 1))

    series_y=normalizers["pm2.5"].transform(series_y.values.reshape(-1, 1))

    return df_x, series_y, normalizers

In [0]:
X_train_norm, y_train_norm, normalizers = minmax_scale(X_train, y_train)
X_valid_norm, y_valid_norm, _ = minmax_scale(X_valid, y_valid, normalizers=normalizers)
X_test_norm, y_test_norm, _ = minmax_scale(X_test, y_test, normalizers=normalizers)

## Encoding of ordinals

The encoding of `cbwd` is interesting, since it is an ordinal again, or better to say not even that, it has a nice circular topology, so we will use the same sin-cos solution.

Problem is, that there is a valid "zero" value, marked "cv" in there. We are tempted to replace that with 0.

In [0]:
def encode_cyclicals(df_x):
    #"month","day","hour", "cdbw", "dayofweek"
    
    DIRECTIONS = {"N":1.0,"NE":2.0, "E":3.0, "SE":4.0, "S":5.0, "SW":6.0, "W":7.0, "NW":8.0, "cv":np.nan}

    df_x['month_sin'] = np.sin(2*np.pi*df_x.month/12)
    df_x['month_cos'] = np.cos(2*np.pi*df_x.month/12)
    df_x.drop('month', axis=1, inplace=True)
    
    df_x['day_sin'] = np.sin(2*np.pi*df_x.day/31)
    df_x['day_cos'] = np.cos(2*np.pi*df_x.day/31)
    df_x.drop('day', axis=1, inplace=True)

    df_x['dayofweek_sin'] = np.sin(2*np.pi*df_x.dayofweek/7)
    df_x['dayofweek_cos'] = np.cos(2*np.pi*df_x.dayofweek/7)
    df_x.drop('dayofweek', axis=1, inplace=True)
    
    df_x['hour_sin'] = np.sin(2*np.pi*df_x.hour/24)
    df_x['hour_cos'] = np.cos(2*np.pi*df_x.hour/24)
    df_x.drop('hour', axis=1, inplace=True)
    
    df_x.replace({'cbwd': DIRECTIONS}, inplace=True)
    df_x['cbwd'] = df_x['cbwd'].astype(np.float64) 

    df_x['cbwd_sin'] = np.sin(2.0*np.pi*df_x.cbwd/8.0)
    df_x['cbwd_sin'].replace(np.nan, 0.0, inplace=True) #Let's handle the case with no wind specially
    df_x['cbwd_cos'] = np.cos(2.0*np.pi*df_x.cbwd/8.0)
    df_x['cbwd_cos'].replace(np.nan, 0.0, inplace=True) #Let's handle the case with no wind specially
    df_x.drop('cbwd', axis=1, inplace=True)
    
    return df_x

In [0]:
X_train_norm = encode_cyclicals(X_train_norm)
X_valid_norm = encode_cyclicals(X_valid_norm)
X_test_norm = encode_cyclicals(X_test_norm)

In [0]:
X_train_norm

In [0]:
#Just in case to ensure we did everything right
assert all(x==np.float64 for x in list(X_train_norm.dtypes))

It would be worth checking with some assertions that the manual normalizers work well. Let's leave it to later work.

It is also worth noting, that the normalizers should be saved and used in production.

# Creating target (y) and "windows" (X) for modeling

By default we use the next 24 hour value of "pm2.5" for prediction, that is, I would like to predict what the pm2.5 will be like **at this hour 24 hours from now.**

We use the quite handy **seglearn** package for this.

Because of computational reasons, we **use the window of 100 hours** to predict. Classical models would have hard time to accommodate substantially (like 5-10x) context windows, LSTM-s would suffer from the challenge of long term memory. After a basic run of modeling the next big challenge would be to investigate PACF structure more and use eg. stateful LSTM modeling to try to accommodate the large "lookback".   

In [0]:
TIME_WINDOW=100
FORECAST_DISTANCE=24

In [0]:
from seglearn.transform import FeatureRep, SegmentXYForecast, last

segmenter = SegmentXYForecast(width=TIME_WINDOW, step=1, y_func=last, forecast=FORECAST_DISTANCE)

X_train_rolled, y_train_rolled,_=segmenter.fit_transform([X_train_norm.values],[y_train_norm.flatten()])

In [0]:
X_train_rolled[:1]

# For non-sequence models 

We have to "flatten" the data to be able to use classical, non-sequence regression models from Scikit.

**We only need to do this for X, any transformation of y is unnecessary.**

In [0]:
X_train_rolled.shape

In [0]:
shape = X_train_rolled.shape
X_train_flattened = X_train_rolled.reshape(shape[0],shape[1]*shape[2])
X_train_flattened.shape

In [0]:
X_valid_rolled, y_valid_rolled,_=segmenter.fit_transform([X_valid_norm.values],[y_valid_norm.flatten()])

shape = X_valid_rolled.shape
X_valid_flattened = X_valid_rolled.reshape(shape[0],shape[1]*shape[2])

# Evaluation helper

Use this function to evaluate your models **on validation data.**

This assumes that your model has the `predict()` function, which is true for **Scikit-learn, XGBoost** and **Keras**, so you can can hand over any of those. 

A special issue by models optimized by iterative methods is to **get the final model**. **Early stopping and / or model save and reload** can help there.  

**WARNING: This is just a basic evaluation scheme, more thorough investigation needed in the future!**

In [0]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def evaluate_model(model, X_valid, y_valid_true):
    predictions = model.predict(X_valid)
    rms = sqrt(mean_squared_error(y_valid_true, predictions))
    print("Root mean squared error on valid:",rms)
    normalized_rms = normalizers["pm2.5"].inverse_transform(np.array([rms]).reshape(1, -1))[0][0]
    print("Root mean squared error on valid inverse transformed from normalization:",normalized_rms)
    return normalized_rms

# Classical modeling

In "classical" modeling we assume a multiple regression case, so we **DO NOT USE time series as such, but the "flat" versions of the data** as input. Output is the same. 

## Baseline - DummyPredictor

**TASK Create a dummy predictor as a baseline. Use Scikit-learn's builtin capability to do dummy models in regression case. Use the default setting, that is the prediction of the mean value.**

In [0]:
from sklearn.dummy import DummyRegressor

dummy_model = DummyRegressor(strategy='mean', constant=None, quantile=None)

dummy_model.fit(X_train_flattened, y_train_rolled)

### Evaluation

In [0]:
result = evaluate_model(dummy_model, X_valid_flattened, y_valid_rolled)

## Fitting a RandomForest on raw data

**TASK: Fit a RandomForest from Scikit. Please be aware, that the number of trees in the model is having a strong influence on training time.** 

**Suggestion:** use couple of tens of trees, definitely << 100 to be able to wait it out...

**Pro tip:** To utilize all the CPU cores, use the right setting of n_jobs. That speeds things up.

In [0]:
from sklearn.ensemble import RandomForestRegressor

N_ESTIMATORS = 10
RANDOM_STATE = 452543634

In [0]:
RF_base_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                      criterion='mse', 
                                      max_depth=None, 
                                      min_samples_split=2, 
                                      min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, 
                                      max_features='auto', 
                                      max_leaf_nodes=None, 
                                      min_impurity_decrease=0.0, 
                                      min_impurity_split=None, 
                                      bootstrap=True, 
                                      oob_score=False, 
                                      n_jobs=None, 
                                      random_state=RANDOM_STATE, 
                                      verbose=0, 
                                      warm_start=False)

RF_base_model.fit(X_train_flattened, y_train_rolled)

### Evaluation


In [0]:
result = evaluate_model(RF_base_model, X_valid_flattened, y_valid_rolled)

## Fitting a RandomForest on feature transformed data

**TASK: Since we use `seglearn`, we can try to capitalize on it's functionality to calculate features from the time time series. Use `FeatureRep` from `seglearn` to transform features, fit a RandomForest and hope for the best!**

In [0]:
# https://dmbee.github.io/seglearn/transform.html

RF_feature_model = RandomForestRegressor(n_estimators=N_ESTIMATORS, 
                                      criterion='mse', 
                                      max_depth=None, 
                                      min_samples_split=2, 
                                      min_samples_leaf=1, 
                                      min_weight_fraction_leaf=0.0, 
                                      max_features='auto', 
                                      max_leaf_nodes=None, 
                                      min_impurity_decrease=0.0, 
                                      min_impurity_split=None, 
                                      bootstrap=True, 
                                      oob_score=False, 
                                      n_jobs=None, 
                                      random_state=RANDOM_STATE, 
                                      verbose=0, 
                                      warm_start=False)

feature_converter = FeatureRep()

RF_feature_model.fit(feature_converter.fit_transform(X_train_flattened), y_train_rolled)

### Evaluation

WARNING: This is just a basic evaluation scheme, more thorough investigation needed!

In [0]:
result = evaluate_model(RF_feature_model,feature_converter.fit_transform(X_train_flattened), y_train_rolled)

## XGBoost for speedup

Use the XGBoost library to fit gradient boosted trees to the problem. They are usually way quicker to learn and many times at least on par with RandomForests, or better. Let's see!

In [0]:
import xgboost as xgb
# If in trouble, use !pip install xgboost

# XGBoost needs it's custom data format to run quickly
dmatrix_train = xgb.DMatrix(data=X_train_flattened,label=y_train_rolled)
dmatrix_valid = xgb.DMatrix(data=X_valid_flattened,label=y_valid_rolled)

In [0]:
params = {'objective': 'reg:linear', 'eval_metric': 'rmse', 'n_estimators': 3}

evallist = [(dmatrix_valid, 'eval'), (dmatrix_train, 'train')]

num_round = 3 #Can easily overfit, experiment with it!

xg_reg = xgb.train(params, dmatrix_train, num_round, evallist, early_stopping_rounds=5)
# https://stackoverflow.com/questions/37650892/xgboost-on-python-what-is-wrong-with-xgb-cv

In [0]:
result = evaluate_model(xg_reg, dmatrix_valid, y_valid_rolled)

# Building an LSTM model

## Modeling assumptions

**TASK:** We believe, that the time dependent structure of this dataset is complex, so we try to use LSTM models from Keras. We are not explicitly utilizing **statefulness**, that is a **major area to be investigated later on**. We use GPU optimizations with the help of [CuDNNLSTM](https://www.tensorflow.org/api_docs/python/tf/keras/layers/CuDNNLSTM) for high speed training.

More information on statefulness can be found [here](https://philipperemy.github.io/keras-stateful-lstm/).


Fit an LSTM model **on the time series - non-flat - data!**.

Use:
1. At least 1 LSTM layer
2. A dense layer for output - think about activation! This is a regression case!

**Very advisable** - but optional - to use Dropout. Sadly the CuDNNLSTM implementation does not have it built in. You can choose: Either use normal LSTM (not really a good idea, just check the speed of learning!) or use CuDNNLSTM and a dropout layer. You can not use it everywhere, though... Experiment!

You are allowed to use functional API, but for this **Sequential API is sufficient.**

You **can use LeraningRateScheduler** if you like.

In [0]:
LSTM_CELL_SIZE = 5
BATCH_SIZE = 500
EPOCHS = 20
DROPOUT_RATE=0.1

In [0]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, CuDNNLSTM
from tensorflow.keras import backend as be
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.models import load_model

column_count = len(X_train.columns) #Remember,column count before rolling...

be.clear_session()

# You might very well be needing it!
# Remeber to save only what is worth it from validation perspective...
# model_saver = ModelCheckpoint(...)

# If you need it...
#def schedule(epoch, lr):
#    ...
#    return lr

#lr_scheduler = LearningRateScheduler(schedule)

# Build your whole LSTM model here!
model = Sequential()

model.add(CuDNNLSTM(units=LSTM_CELL_SIZE, input_shape=(TIME_WINDOW, column_count)))

model.add(Dense(1, activation = 'linear'))

model.compile(optimizer='adadelta', loss='mean_squared_error')

#For shape remember, we have a variable defining the "window" and the features in the window...

# Fit on the train data
# USE the batch size parameter!
# Use validation data - warning, a tuple of stuff!
# Epochs as deemed necessary...
# You should avoid shuffling the data maybe.
# You can use the callbacks for LR schedule or model saving as seems fit.
history = model.fit(X_train_rolled, y_train_rolled, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(X_valid_rolled, y_valid_rolled), class_weight=None, sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None)

In [0]:
print(history.history)

plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

In [0]:
# You can use the early stopped model OR load it. 
# For that you have to import the load function...
# IF AND ONLY IF loading, it is good practice to throw out the trash from the graph...
# be.clear_session()

result = evaluate_model(model, X_valid_rolled, y_valid_rolled)

In [0]:
assert result < 86.0

# Things that should be improved

- More conclusive investigation of PACF for better time window estimate
    - It can well be, that long windows do not add that much to the performance
- More interesting features for XGBoost (like from [tsfresh](https://tsfresh.readthedocs.io/en/latest/)), since present features are a disaster
- MOST IMPORTANT: **More thorough error / prediction analysis!!!**
- LSTM with **Custom iterator with stateful model**
- Investigation of different loss function (eg. MAE) for training. (And with it, think abut the importance of extreme values: do we think they are outliers? Are they interesing to predict?)
- Investigation of "teacher forcing" for LSTM-s in Keras (if it makes sense)

# Conclusion

Even with decent amount of struggle, the "dummy" of always using the mean is very appealing, so it seems, this is not that easy of a task 24 hours in advance. Further investigation of classical as well as neural models remains open!


# Final test

We did not use the final test, since our investigations are not concluded yet. Remember: using it once before project "go live" is a good practice!